In [18]:
import numpy as np
import scipy.io as sio
import os
import pandas as pd

In [19]:
# Matlab file path
METADATA_FILE_MATLAB = '../dataset/FDDB/dataset_labels_multibox.mat'

In [20]:
mat_obj = sio.loadmat(METADATA_FILE_MATLAB)
mat_dta = mat_obj['data2']


In [21]:
paths = []
bboxes = []
for path, bbox in mat_dta:
    # Path
    norm_path = os.path.join('..', '..', 'dataset', *path[0].split('\\'))
    paths.append(norm_path)
    # Bounding boxes
    bbox_list = []
    for b in bbox.flatten():
        rounded_list = [round(x) for x in list(b.flatten())]
        bbox_list.append(rounded_list)
    bboxes.append(bbox_list)

In [22]:
lines = []

pd.DataFrame(columns=['filename', 'bboxes'])
bboxes_list = []
for f_name, b in zip(paths, bboxes):
    fix_box = []
    for box in b:
        fix_box.append([box[0], box[1], box[0]+box[2], box[1]+box[3]])
    bboxes_list.append(fix_box)
    print()
    all_bbox = ' '.join([','.join([str(s) for s in x] + [str(0)]) for x in fix_box])
    row = f'{f_name} {all_bbox}'
    lines.append(row)


In [35]:
df = pd.DataFrame(columns=['path', 'bboxes'])
df['path'] = [x.split('FDDB')[-1].lstrip('/') for x in paths]
df['bboxes'] = bboxes_list
df.head()

,path,bboxes
0,originalPics/2002/08/11/big/img_591.jpg,"[[180, 41, 359, 282]]"
1,originalPics/2002/08/26/big/img_265.jpg,"[[60, 20, 150, 154], [157, 88, 211, 172], [296..."
2,originalPics/2002/07/19/big/img_423.jpg,"[[196, 47, 315, 221]]"
3,originalPics/2002/08/24/big/img_490.jpg,"[[110, 24, 182, 132]]"
4,originalPics/2002/08/31/big/img_17676.jpg,"[[1, 1, 58, 75], [62, 13, 163, 172]]"


In [36]:
df.to_pickle('../dataset/FDDB/fddb_metadata.pickle')

In [38]:
with open('../libs/yolo3_keras/model_data/fddb_annotation.txt', 'w') as f:
    f.write('\n'.join(lines))